In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/contradictory-my-dear-watson/sample_submission.csv
/kaggle/input/contradictory-my-dear-watson/train.csv
/kaggle/input/contradictory-my-dear-watson/test.csv


In [2]:
# ライブラリのインポート
from sklearn.model_selection import train_test_split
from transformers import AdamWeightDecay, AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf

2021-11-08 08:18:02.725626: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-11-08 08:18:02.725748: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
# TPUとの接続、初期化
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
    print('Number of replicas:', strategy.num_replicas_in_sync)
except ValueError:
    strategy = tf.distribute.get_strategy() # for CPU and single GPU
    print('Number of replicas:', strategy.num_replicas_in_sync)

2021-11-08 08:18:09.094732: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-08 08:18:09.097750: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2021-11-08 08:18:09.097785: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-08 08:18:09.097811: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (55515fcd7d7f): /proc/driver/nvidia/version does not exist
2021-11-08 08:18:09.101595: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

Number of replicas: 8


In [4]:
# ハイパーパラメータの定義、トークナイザの初期化
model_name = "bert-base-multilingual-cased"
epochs = 100
max_length = 120

batch_size = 16 * strategy.num_replicas_in_sync
print(f"batch_size: {batch_size}")

auto = tf.data.experimental.AUTOTUNE
tokenizer = AutoTokenizer.from_pretrained(model_name)

batch_size: 128


Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

In [5]:
# 学習データの読み込み
all_train = pd.read_csv("../input/contradictory-my-dear-watson/train.csv")

# 交差検証のためにデータを分割
train, valid = train_test_split(all_train, test_size=0.2, random_state=0)

# 前提、仮説のテキストのみを抽出
train_text = train[['premise', 'hypothesis']].values.tolist()
valid_text = valid[['premise', 'hypothesis']].values.tolist()

# トークナイズ + インデキシング
train_encoded = tokenizer.batch_encode_plus(train_text, padding=True, max_length=max_length, truncation=True)
valid_encoded = tokenizer.batch_encode_plus(valid_text, padding=True, max_length=max_length, truncation=True)

# TensorFlowのDatasetに変換
train_dataset = (
    tf.data.Dataset.from_tensor_slices((dict(train_encoded), train.label.values)).repeat().shuffle(2048).batch(batch_size).prefetch(auto)
)
valid_dataset = (
    tf.data.Dataset.from_tensor_slices((dict(valid_encoded), valid.label.values)).batch(batch_size).cache().prefetch(auto)
)

In [6]:
# strategyスコープ内でモデルをインスタンス化することで、TPU上にモデルが作られる
with strategy.scope():
    model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
    model.compile(
        optimizer=AdamWeightDecay(learning_rate=1e-5),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=tf.metrics.SparseCategoricalAccuracy(),
    )
    model.summary()

Downloading:   0%|          | 0.00/1.08G [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  177853440 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  2307      
Total params: 177,855,747
Trainable params: 177,855,747
Non-trainable params: 0
_________________________________________________________________


In [7]:
# モデル保存用のコールバック。val_lossが最小のときにモデルの重みを保存する
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    f'watson-{model_name}.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=True, mode='min'
)
# 学習を早期終了させるためのコールバック。val_lossが2度連続して改善されなかった場合に学習を終了させる
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='auto')

# モデルの学習
model.fit(
    train_dataset,
    steps_per_epoch=len(train) // batch_size,
    epochs=epochs,
    validation_data=valid_dataset,
    callbacks = [checkpoint, early_stopping]
)

Epoch 1/100
75/75 [==============================] - 89s 308ms/step - loss: 1.0800 - sparse_categorical_accuracy: 0.4074 - val_loss: 0.9614 - val_sparse_categorical_accuracy: 0.5177
Epoch 2/100
75/75 [==============================] - 8s 112ms/step - loss: 0.9039 - sparse_categorical_accuracy: 0.5887 - val_loss: 0.8509 - val_sparse_categorical_accuracy: 0.6229
Epoch 3/100
75/75 [==============================] - 10s 128ms/step - loss: 0.7400 - sparse_categorical_accuracy: 0.6858 - val_loss: 0.8505 - val_sparse_categorical_accuracy: 0.6320
Epoch 4/100
75/75 [==============================] - 8s 111ms/step - loss: 0.6268 - sparse_categorical_accuracy: 0.7439 - val_loss: 0.8879 - val_sparse_categorical_accuracy: 0.6333
Epoch 5/100
75/75 [==============================] - 8s 111ms/step - loss: 0.5216 - sparse_categorical_accuracy: 0.7991 - val_loss: 0.9371 - val_sparse_categorical_accuracy: 0.6399
